# Getting necessary features for file:
# gcbu_dnc_recompletions
Here we will be isolating a couple features along with separating the data out by different rig company and eventually find the average of each feature per rig company

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

import datetime

from dateutil.relativedelta import relativedelta

from datetime import date
import numpy as np
import seaborn as sns
import os
import missingno as msno

## Load in the file

In [2]:
path = "Clean_Files"
os.makedirs(path, exist_ok = True)

save_path = path + "/"
# notice order of files
files = [
    "clean_top_cross_bu_dnc_stim_stage_detail.xlsx",
    "clean_bot_cross_bu_dnc_stim_stage_detail.xlsx",
    "clean_gcbu_dnc_recompletions.xlsx",
    "clean_gcbu_dnc_stim_stage.xlsx",
    "clean_monthly_production.xlsx"
]

In [3]:
file = 2

df = pd.read_excel(save_path + files[file])
production = pd.read_excel(save_path + files[-1])

In [ ]:
df.columns

In [4]:
# here due to the fact that my file did not contian the commented out files
# I just went ahead and replaced them with their STIM_INT counter parts.
# That could be an error to do, but for now I'll stick with it.

wanted_features = [
    "UWI",
    "RIG_ID",
    "RIG_ACCEPT_DATE",
    "RIG_RELEASE_DATE",
    "STG_START_DATE",        
    "STG_END_DATE",            
    "STG_SLURRY_VOLUME_PUMPED",
    "PROPPANT_DESIGNED",        
    "STG_TOP_DEPTH",
    "STG_BOTTOM_DEPTH",
]

In [5]:
# Remove those features that I wanted

working = df[wanted_features]

In [ ]:
working

In [6]:
# Here because I did not want to work with an entry that did not have a rig
# contractor id, I just dropped it.

working = working.dropna(subset = ["RIG_ID"])

In [ ]:
working = working.reset_index(drop = True)
working

In [7]:
# Converting all dates so that they will be easier to work with later.

dates = [
    "STG_START_DATE",
    "STG_END_DATE",
    "RIG_ACCEPT_DATE",
    "RIG_RELEASE_DATE"
]

for d in dates:
    working[d] = pd.to_datetime(working[d], dayfirst=True)

In [8]:
# Getting rid of the "fluff" that is infront of the rig contractor id
# so I'm re working the row so that only the id is there

for i in range(0, len(working)):
    working.loc[i, "RIG_ID"] = working.loc[i, "RIG_ID"].split(" ")[2]

In [ ]:
working

## Finding interpolated variables

In [9]:
inter_vars = [
    "Rig_Duration (days)",
    "Stage_Duration (hrs)",
    "Stage_Length (meters)",
    "Production (monthly)",
]

In [10]:
# Here we are simultaniously finding the rig duration and the stim_int duration
# So the process is that, if one of the dates is invalid (i.e. just missing)
# just write it as -1.

# If both dates are valid, but the release/end date is before the start date
# then put -1.

# Else, put in the correct duration in either days or in hours.

for i in range(0, len(working)):
    start_rig = working.loc[i, "RIG_ACCEPT_DATE"]
    end_rig = working.loc[i, "RIG_RELEASE_DATE"]
    if(pd.isnull(start_rig) or pd.isnull(end_rig)):
        working.loc[i, inter_vars[0]] = -1
    else:
        temp = (end_rig - start_rig).components.days
        if(temp < 0):
            working.loc[i, inter_vars[0]] = -1
        else:
            working.loc[i, inter_vars[0]] = temp
    
    
    start_stim = working.loc[i, "STG_START_DATE"]
    end_stim = working.loc[i, "STG_END_DATE"]
    if(pd.isnull(start_stim) or pd.isnull(end_stim)):
        working.loc[i, inter_vars[1]] = -1
    else:
        temp = (end_stim - start_stim).components.hours + 24 * (end_stim - start_stim).components.days
        if(temp < 0):
            working.loc[i, inter_vars[1]] = -1
        else:
            working.loc[i, inter_vars[1]] = temp

In [11]:
# Next find stage lenght for each entry
working[inter_vars[2]] = working["STG_BOTTOM_DEPTH"] - working["STG_TOP_DEPTH"]

In [12]:
working

,UWI,RIG_ID,RIG_ACCEPT_DATE,RIG_RELEASE_DATE,STG_START_DATE,STG_END_DATE,STG_SLURRY_VOLUME_PUMPED,PROPPANT_DESIGNED,STG_TOP_DEPTH,STG_BOTTOM_DEPTH,Rig_Duration (days),Stage_Duration (hrs),Stage_Length (meters)
0,2019000699,2019000082,2019-01-27 06:00:00,2019-03-02 18:00:00,2019-01-28 08:05:00,2019-01-28 10:05:00,4632,152125.0,17968,18041,34.0,2.0,73
1,2019000665,2019000098,2018-02-14 08:00:00,2018-02-23 18:00:00,2018-02-19 14:40:00,2018-02-20 00:19:00,24452,0.0,13063,18327,9.0,9.0,5264
2,2019000347,2019000084,2017-10-25 06:00:00,2017-10-29 06:00:00,2017-10-26 10:13:00,2017-10-26 12:45:00,5043,0.0,12103,16589,4.0,2.0,4486
3,2019001412,2019000098,2019-03-24 18:00:00,2019-03-30 06:00:00,2019-03-25 14:05:00,2019-03-25 15:34:00,3015,130000.0,18290,18355,5.0,1.0,65
4,2019001390,2019000098,2019-06-06 06:00:00,2019-06-15 06:00:00,2019-06-13 07:17:00,2019-06-13 09:19:00,7677,366938.0,13694,13842,9.0,2.0,148
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1649,2019000628,2019000098,2019-07-20 06:00:00,2019-02-08 18:00:00,2019-01-08 15:26:00,2019-01-08 16:21:00,4047,192000.0,11745,11825,-1.0,0.0,80
1650,2019000628,2019000098,2019-07-20 06:00:00,2019-02-08 18:00:00,2019-01-08 18:21:00,2019-01-08 19:23:00,4172,192000.0,11650,11730,-1.0,1.0,80
1651,2019000628,2019000098,2019-07-20 06:00:00,2019-02-08 18:00:00,2019-01-08 21:27:00,2019-01-08 22:29:00,4195,192000.0,11555,11635,-1.0,1.0,80
1652,2019000628,2019000098,2019-07-20 06:00:00,2019-02-08 18:00:00,2019-02-08 00:30:00,2019-02-08 01:31:00,4057,192000.0,11460,11540,-1.0,1.0,80


## Production stuff

In [16]:
# the way this method works is that, given a start and an end date
# the function will return a list of tuples that represent all the months and years
# that a given well operated for. The end month is included in the list.

# Example, if start date was 1/1/20 and end date was 3/1/20, the list that will
# return is: [(1,2020), (2,2020), (3,2020)]

# If an invalid date is given then it will return an empty list.
# If an invalid date range (end comes before start) an empty list will be returned.

def all_dates(start, end):
    to_return = []
    
    if(pd.isnull(start) or pd.isnull(end)):
        return to_return
    
    start_year = start.year
    start_month = start.month
    
    end_year = end.year
    end_month = end.month
    
    if(end_year < start_year):
        return to_return
    elif(end_year == start_year and end_month < start_month):
        return to_return
    
    '''
    if(end_month+1 == 12):
        end_month = 1
        end_year += 1
    if(end_month+1 == 13):
        end_month = 2
        end_year += 1
    ''' 
    while(start_year != end_year or start_month != end_month):
        to_return.append((start_month, start_year))
        start_month += 1
        if(start_month == 13):
            start_month = 1
            start_year += 1
    to_return.append((start_month,start_year))
    
    return to_return

In [23]:
# This method will return the total oil production of a given well along
# a list of dates.

# if no production is found, then -1 will be put.
def find_production(uwi, dates):
    total_prod = 0
    
    for date in dates:
        m = date[0]
        y = date[1]
        
        temp = production[(production["UWI"] == uwi) & 
                          (production["CYCLE_YEAR"] == y) & 
                          (production["CYCLE_MONTH"]== m)]
        
        if(len(temp) == 0):
            total_prod += 0
        else:
            total_prod += temp["LEASE_OIL_PROD_VOL"].sum()

            #total_prod += temp.loc[0, "LEASE_OIL_PROD_VOL"]
    
    if(total_prod == 0):
        return -1
    else:
        return total_prod
        
        

In [32]:
#working[['UWI','RIG_ACCEPT_DATE','RIG_RELEASE_DATE','Rig_Duration (days)']].sample(10)
#working[working['UWI']==2019001396]

In [39]:
strt = working.loc[679, "RIG_ACCEPT_DATE"]
end = working.loc[679, "RIG_RELEASE_DATE"]
w_id = working.loc[679, "UWI"]

d = all_dates(strt,end)
total = 0
for ds in d:
    m = ds[0]
    y = ds[1]
    temp = production[(production["UWI"] == w_id) & (production["CYCLE_YEAR"] == y) & (production["CYCLE_MONTH"]== m)]
    #print(temp)
#print(total)
    total += temp['LEASE_OIL_PROD_VOL'].sum()
#production[production['UWI'] == w_id]
#print(total)
#temp
prod = find_production(w_id,d)
test = working.copy()
test.loc[679,'test'] = prod
test

,UWI,RIG_ID,RIG_ACCEPT_DATE,RIG_RELEASE_DATE,STG_START_DATE,STG_END_DATE,STG_SLURRY_VOLUME_PUMPED,PROPPANT_DESIGNED,STG_TOP_DEPTH,STG_BOTTOM_DEPTH,Rig_Duration (days),Stage_Duration (hrs),Stage_Length (meters),Oil Production (total),test
0,2019000699,2019000082,2019-01-27 06:00:00,2019-03-02 18:00:00,2019-01-28 08:05:00,2019-01-28 10:05:00,4632,152125.0,17968,18041,34.0,2.0,73,-1,NaN
1,2019000665,2019000098,2018-02-14 08:00:00,2018-02-23 18:00:00,2018-02-19 14:40:00,2018-02-20 00:19:00,24452,0.0,13063,18327,9.0,9.0,5264,-1,NaN
2,2019000347,2019000084,2017-10-25 06:00:00,2017-10-29 06:00:00,2017-10-26 10:13:00,2017-10-26 12:45:00,5043,0.0,12103,16589,4.0,2.0,4486,-1,NaN
3,2019001412,2019000098,2019-03-24 18:00:00,2019-03-30 06:00:00,2019-03-25 14:05:00,2019-03-25 15:34:00,3015,130000.0,18290,18355,5.0,1.0,65,-1,NaN
4,2019001390,2019000098,2019-06-06 06:00:00,2019-06-15 06:00:00,2019-06-13 07:17:00,2019-06-13 09:19:00,7677,366938.0,13694,13842,9.0,2.0,148,-1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1649,2019000628,2019000098,2019-07-20 06:00:00,2019-02-08 18:00:00,2019-01-08 15:26:00,2019-01-08 16:21:00,4047,192000.0,11745,11825,-1.0,0.0,80,-1,NaN
1650,2019000628,2019000098,2019-07-20 06:00:00,2019-02-08 18:00:00,2019-01-08 18:21:00,2019-01-08 19:23:00,4172,192000.0,11650,11730,-1.0,1.0,80,-1,NaN
1651,2019000628,2019000098,2019-07-20 06:00:00,2019-02-08 18:00:00,2019-01-08 21:27:00,2019-01-08 22:29:00,4195,192000.0,11555,11635,-1.0,1.0,80,-1,NaN
1652,2019000628,2019000098,2019-07-20 06:00:00,2019-02-08 18:00:00,2019-02-08 00:30:00,2019-02-08 01:31:00,4057,192000.0,11460,11540,-1.0,1.0,80,-1,NaN


In [42]:
# Doing all this process to get the correct production for each entry
for i in range(0, len(working)):
    strt = working.loc[i, "RIG_ACCEPT_DATE"]
    end = working.loc[i, "RIG_RELEASE_DATE"]
    w_id = working.loc[i, "UWI"]
    rig_dur = working.loc[i, "Rig_Duration (days)"]
    
    if(rig_dur != -1):
        d = all_dates(strt, end)
        prod = find_production(w_id, d)
        working.loc[i,"Oil Production (total)"] = prod
    else:
        working.loc[i,"Oil Production (total)"] = -1

In [43]:
working[working['Oil Production (total)'] != -1]

,UWI,RIG_ID,RIG_ACCEPT_DATE,RIG_RELEASE_DATE,STG_START_DATE,STG_END_DATE,STG_SLURRY_VOLUME_PUMPED,PROPPANT_DESIGNED,STG_TOP_DEPTH,STG_BOTTOM_DEPTH,Rig_Duration (days),Stage_Duration (hrs),Stage_Length (meters),Oil Production (total)
0,2019000699,2019000082,2019-01-27 06:00:00,2019-03-02 18:00:00,2019-01-28 08:05:00,2019-01-28 10:05:00,4632,152125.0,17968,18041,34.0,2.0,73,25248.0
3,2019001412,2019000098,2019-03-24 18:00:00,2019-03-30 06:00:00,2019-03-25 14:05:00,2019-03-25 15:34:00,3015,130000.0,18290,18355,5.0,1.0,65,45614.0
6,2019000343,2019000098,2019-07-09 06:00:00,2019-11-09 06:00:00,2019-09-09 11:37:00,2019-09-09 14:20:00,7301,384000.0,16612,16760,123.0,2.0,148,304.0
11,2019001396,2019000098,2019-03-16 06:00:00,2019-03-25 06:00:00,2019-03-17 04:17:00,2019-03-17 09:07:00,9862,390000.0,18092,18183,9.0,4.0,91,5070.0
18,2019001277,2019000081,2018-07-18 18:00:00,2018-07-28 06:00:00,2018-07-20 02:07:00,2018-07-20 04:52:00,8363,325000.0,16559,16648,9.0,2.0,89,355.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1434,2019001325,2019000021,2019-01-18 12:00:00,2019-01-25 06:00:00,2019-01-23 11:58:00,2019-01-23 13:52:00,6645,320000.0,14082,14225,6.0,1.0,143,1.0
1439,2019001325,2019000021,2019-01-18 12:00:00,2019-01-25 06:00:00,2019-01-23 16:29:00,2019-01-23 18:21:00,6637,320000.0,13919,14062,6.0,1.0,143,1.0
1444,2019001325,2019000021,2019-01-18 12:00:00,2019-01-25 06:00:00,2019-01-23 22:38:00,2019-01-24 00:33:00,6891,320000.0,13757,13900,6.0,1.0,143,1.0
1449,2019001325,2019000021,2019-01-18 12:00:00,2019-01-25 06:00:00,2019-01-24 02:55:00,2019-01-24 04:39:00,6554,320000.0,13595,13738,6.0,1.0,143,1.0


In [44]:
# Saving off the working file

working.to_excel(save_path + "working_file_" + files[file], index = False)

In [47]:
# reduce the data frame to stuff we will use. So all the dates and the well
# ids are no longer needed.

final_features = [
    "RIG_ID",
    "STG_TOP_DEPTH",
    "STG_BOTTOM_DEPTH",
    "STG_SLURRY_VOLUME_PUMPED",
    "PROPPANT_DESIGNED",
    "Rig_Duration (days)",
    "Stage_Duration (hrs)",
    "Stage_Length (meters)",
]

In [48]:
final_df = working[final_features]

In [49]:
final_df

,RIG_ID,STG_TOP_DEPTH,STG_BOTTOM_DEPTH,STG_SLURRY_VOLUME_PUMPED,PROPPANT_DESIGNED,Rig_Duration (days),Stage_Duration (hrs),Stage_Length (meters)
0,2019000082,17968,18041,4632,152125.0,34.0,2.0,73
1,2019000098,13063,18327,24452,0.0,9.0,9.0,5264
2,2019000084,12103,16589,5043,0.0,4.0,2.0,4486
3,2019000098,18290,18355,3015,130000.0,5.0,1.0,65
4,2019000098,13694,13842,7677,366938.0,9.0,2.0,148
...,...,...,...,...,...,...,...,...
1649,2019000098,11745,11825,4047,192000.0,-1.0,0.0,80
1650,2019000098,11650,11730,4172,192000.0,-1.0,1.0,80
1651,2019000098,11555,11635,4195,192000.0,-1.0,1.0,80
1652,2019000098,11460,11540,4057,192000.0,-1.0,1.0,80


In [50]:
# This is just to find out how many unique rig contractors there are

unique_rigs = final_df["RIG_ID"].unique()
print(len(unique_rigs))
print(unique_rigs)

7
['2019000082' '2019000098' '2019000084' '2019000081' '2019000085'
 '2019000021' '2019000166']


In [51]:
all_rigs = []

for rig_id in unique_rigs:
    temp_rig = final_df[final_df["RIG_ID"] == rig_id].reset_index(drop = True)
    all_rigs.append(temp_rig)

In [52]:
# Here we will be standardizing every single entry so that we don't have
# to find a way to make the grades easier to calculate

# The formula that we'll be using is:

# standard value = abs(value - mean) / std
# except in the case that std is either null or 0, in those cases
# treat the std as 1 and continue with the formula

for rig in all_rigs:
    temp_describe = rig.describe()
    for feature in rig.columns.drop("RIG_ID"):
        mean = temp_describe.loc["mean", feature]
        std = temp_describe.loc["std", feature]
        if(pd.isnull(std) or std == 0):
            std = 1
        for i in range(0, len(rig)):
            rig.loc[i, feature] = abs(rig.loc[i, feature] - mean) / std

In [54]:
statistics = pd.DataFrame(
    columns = final_df.columns.drop("RIG_ID").insert(0, "RIG_NUMBER").insert(1, "Stages Completed").drop("STG_TOP_DEPTH").drop("STG_BOTTOM_DEPTH")
)
statistics

,RIG_NUMBER,Stages Completed,STG_SLURRY_VOLUME_PUMPED,PROPPANT_DESIGNED,Rig_Duration (days),Stage_Duration (hrs),Stage_Length (meters)


In [57]:
# Add in the statistics for each rig into a final statistics dataframe
for rig in range(0, len(all_rigs)):
    temp_row = {
        'RIG_NUMBER':                          "rig_%d" % (rig),
        'Stages Completed':                    len(all_rigs[rig]),
        'STG_SLURRY_VOLUME_PUMPED':       all_rigs[rig].describe().loc["mean", "STG_SLURRY_VOLUME_PUMPED"],
        'PROPPANT_DESIGNED':          all_rigs[rig].describe().loc["mean", "PROPPANT_DESIGNED"],
        'Rig_Duration (days)':                 all_rigs[rig].describe().loc["mean", "Rig_Duration (days)"],
        'Stage_Duration (hrs)':             all_rigs[rig].describe().loc["mean", "Stage_Duration (hrs)"],
        'Stage_Length (meters)':               all_rigs[rig].describe().loc["mean", "Stage_Length (meters)"],
    }
    statistics = statistics.append(temp_row, ignore_index = True)

In [58]:
statistics

,RIG_NUMBER,Stages Completed,STG_SLURRY_VOLUME_PUMPED,PROPPANT_DESIGNED,Rig_Duration (days),Stage_Duration (hrs),Stage_Length (meters)
0,rig_0,228,0.505194,0.537572,0.580810,0.388346,0.983232
1,rig_1,403,0.726306,0.785023,0.553973,0.278455,0.676504
2,rig_2,162,0.495710,0.905771,0.920188,0.309343,0.840721
3,rig_3,415,0.460409,0.773619,0.628910,0.218249,0.912761
4,rig_4,165,0.462212,0.736381,0.754017,0.266384,0.663172
5,rig_5,259,0.488834,0.673565,0.587652,0.213537,0.967747
6,rig_6,22,0.406248,0.407020,0.000000,0.575797,0.403910


In [59]:
statistics = statistics.set_index("RIG_NUMBER")
statistics

,Stages Completed,STG_SLURRY_VOLUME_PUMPED,PROPPANT_DESIGNED,Rig_Duration (days),Stage_Duration (hrs),Stage_Length (meters)
RIG_NUMBER,,,,,,
rig_0,228,0.505194,0.537572,0.580810,0.388346,0.983232
rig_1,403,0.726306,0.785023,0.553973,0.278455,0.676504
rig_2,162,0.495710,0.905771,0.920188,0.309343,0.840721
rig_3,415,0.460409,0.773619,0.628910,0.218249,0.912761
rig_4,165,0.462212,0.736381,0.754017,0.266384,0.663172
rig_5,259,0.488834,0.673565,0.587652,0.213537,0.967747
rig_6,22,0.406248,0.407020,0.000000,0.575797,0.403910


In [60]:
# Calculating the grade

for rig in statistics.index:
    v_grade = 0.2 * statistics.loc[rig, "STG_SLURRY_VOLUME_PUMPED"]
    p_grade = 0.2 * statistics.loc[rig, "PROPPANT_DESIGNED"]
    r_grade = 0.1 * statistics.loc[rig, "Rig_Duration (days)"]
    s_grade = 0.35 * statistics.loc[rig, "Stage_Duration (hrs)"]
    l_grade = 0.15 * statistics.loc[rig, "Stage_Length (meters)"]
    statistics.loc[rig, "Grade"] = v_grade + p_grade + r_grade + s_grade + l_grade

In [61]:
statistics = statistics.sort_values(by = ["Grade"], ascending = False)
statistics

,Stages Completed,STG_SLURRY_VOLUME_PUMPED,PROPPANT_DESIGNED,Rig_Duration (days),Stage_Duration (hrs),Stage_Length (meters),Grade
RIG_NUMBER,,,,,,,
rig_2,162,0.495710,0.905771,0.920188,0.309343,0.840721,0.606693
rig_1,403,0.726306,0.785023,0.553973,0.278455,0.676504,0.556598
rig_0,228,0.505194,0.537572,0.580810,0.388346,0.983232,0.550040
rig_3,415,0.460409,0.773619,0.628910,0.218249,0.912761,0.522998
rig_5,259,0.488834,0.673565,0.587652,0.213537,0.967747,0.511145
rig_4,165,0.462212,0.736381,0.754017,0.266384,0.663172,0.507831
rig_6,22,0.406248,0.407020,0.000000,0.575797,0.403910,0.424769


In [62]:
statistics.to_excel(save_path + "grades_" + files[file])